# Transfer Learning

<https://www.kaggle.com/aryan7781/transfer-model-vgg-16>

In [1]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Desktop/hub/datathon2021/Scripts/data/test/Transfer_Learning')
EPOCHS = 3
BATCH_SIZE = 800

## Import dataset

In [2]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape)

(50000, 32, 32, 3)


## Change dim

In [3]:
import cv2
import numpy as np

def gray_to_rgb(X):
    """Expands the array to match the shape of a RGB image"""
    X = map(lambda x: np.stack((x,)*3, axis=-1), X)
    return np.array(list(X))

def resize_array(X, new_size=(32,32)):
    """Resize the array to match the size required from pre-trained model"""
    X = np.array([cv2.resize(x, dsize=new_size, interpolation=cv2.INTER_CUBIC) for x in X])
    return X

In [4]:
from tensorflow.keras.utils import to_categorical

# Reshape the target to be an array of 0 and 1 where the class is present
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape)

(50000, 10)


## Model

### VGG16

In [6]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.applications import vgg16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, GRU

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Get pretrained arquitecture and weights
pretrained_model = vgg16.VGG16(include_top=False, input_shape=X_train.shape[1:], pooling='max')
pretrained_model.trainable = False

# Set the input
inputs = pretrained_model.input

# Add layers to fine-tuning (learn from your data)
x = Dense(128, activation='relu')(pretrained_model.output)
x = Dense(128, activation='relu')(x)

# If there's a categorical task, final layer should have N units where N is the number of classes
outputs = Dense(y_train.shape[-1], activation='softmax')(x)

# Set the model, from inputs to outputs
model = Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
checkpoint = ModelCheckpoint(BASE_DIR.joinpath('models','transfer_vgg16_{val_accuracy:.3f}.h5'),
                             save_best_only=True,
                             save_weights_only=False,
                             monitor='val_accuracy')

# There shoul be a train and test generator
training_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/4
34/34 [==============================] - 190s 6s/step - loss: 5.4746 - accuracy: 0.3180 - val_loss: 2.4861 - val_accuracy: 0.3884
Epoch 2/4
34/34 [==============================] - 223s 7s/step - loss: 2.0670 - accuracy: 0.4222 - val_loss: 1.8567 - val_accuracy: 0.4370
Epoch 3/4
34/34 [==============================] - 262s 8s/step - loss: 1.6531 - accuracy: 0.4771 - val_loss: 1.6583 - val_accuracy: 0.4690
Epoch 4/4
34/34 [==============================] - 267s 8s/step - loss: 1.4673 - accuracy: 0.5152 - val_loss: 1.5500 - val_accuracy: 0.4988


### ResNet50

In [7]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.applications import resnet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, GRU

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Get pretrained arquitecture and weights
pretrained_model = resnet50.ResNet50(include_top=False, input_shape=X_train.shape[1:], pooling='max')
pretrained_model.trainable = False

# Set the input
inputs = pretrained_model.input

# Add layers to fine-tuning (learn from your data)
x = Dense(128, activation='relu')(pretrained_model.output)
x = Dense(128, activation='relu')(x)

# If there's a categorical task, final layer should have N units where N is the number of classes
outputs = Dense(y_train.shape[-1], activation='softmax')(x)

# Set the model, from inputs to outputs
model = Model(inputs=inputs, outputs=outputs)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
checkpoint = ModelCheckpoint(BASE_DIR.joinpath('models','transfer_resnet50_{val_accuracy:.3f}.h5'),
                             save_best_only=True,
                             save_weights_only=False,
                             monitor='val_accuracy')

# There shoul be a train and test generator
training_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/4
67/67 [==============================] - 100s 1s/step - loss: 1.5370 - accuracy: 0.4724 - val_loss: 1.2285 - val_accuracy: 0.5720


/Users/efraflores/Desktop/hub/datathon2021/venv/lib/python3.9/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/4
67/67 [==============================] - 119s 2s/step - loss: 1.1244 - accuracy: 0.6046 - val_loss: 1.1440 - val_accuracy: 0.5983
Epoch 3/4
67/67 [==============================] - 95s 1s/step - loss: 1.0086 - accuracy: 0.6463 - val_loss: 1.1037 - val_accuracy: 0.6163
Epoch 4/4
67/67 [==============================] - 117s 2s/step - loss: 0.9248 - accuracy: 0.6737 - val_loss: 1.1006 - val_accuracy: 0.6150


## Fine Tuning

In [6]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# If there's a categorical task, final layer should have N units where N is the number of classes
outputs = Dense(y_train.shape[-1], activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=outputs)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Resize the array to match the size required from pre-trained model
X_train = resize_array(X_train, new_size=(100,100))
X_test = resize_array(X_test, new_size=(100,100))

# train the model on the new data for a few epochs
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

# at this point, the top layers are well trained and we can start fine-tuning convolutional layers from inception V3
# We will freeze the bottom N layers and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
checkpoint = ModelCheckpoint(BASE_DIR.joinpath('models','transfer_inception_{val_accuracy:.3f}.h5'),
                             save_best_only=True,
                             save_weights_only=False,
                             monitor='val_accuracy')

# we train our model again (this time fine-tuning the top 2 inception blocks alongside the top Dense layers
training_history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

Epoch 1/3
63/63 [==============================] - 230s 4s/step - loss: 175.1464 - accuracy: 0.1421
Epoch 2/3
63/63 [==============================] - 251s 4s/step - loss: 32.6049 - accuracy: 0.1536
Epoch 3/3
63/63 [==============================] - 250s 4s/step - loss: 16.4012 - accuracy: 0.1578
Epoch 1/3
63/63 [==============================] - 512s 8s/step - loss: 2.2968 - accuracy: 0.1168 - val_loss: 2.2645 - val_accuracy: 0.2007
Epoch 2/3
 3/63 [>.............................] - ETA: 15:45 - loss: 2.2918 - accuracy: 0.1229

KeyboardInterrupt: 

## Transfer Tricks

### Extract features

In [8]:
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.preprocessing import image
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

model = VGG16(weights='imagenet', include_top=False)

img_path = str(BASE_DIR.parent.joinpath('Grogu.jpeg'))
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

print(features.shape)

(1, 7, 7, 512)


### Predict classes

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

model = ResNet50(weights='imagenet')

img_path = str(BASE_DIR.parent.joinpath('elephant.jpeg'))
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n01871265', 'tusker', 0.5695967), ('n02504458', 'African_elephant', 0.39313364), ('n02504013', 'Indian_elephant', 0.028914398)]
